In [3]:
from joblib import load
import pandas as pd

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, f1_score

In [5]:
df = pd.read_csv("../data/dataset_cleaned.csv", index_col=0)
X, y = df.drop(columns="attack"), df["attack"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
rf = load("random_forest_model")

## Feature Importance:

In [6]:
importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print(importances.head(40))

system.cpu.dcache.demandMissRate::total              0.059630
system.cpu.dcache.overallMissRate::total             0.042275
system.cpu.dcache.overallMissRate::cpu.data          0.041848
system.cpu.dcache.demandMissRate::cpu.data           0.034587
system.cpu.dcache.demandMshrMissRate::cpu.data       0.029930
system.mem_ctrl.avgPriority_cpu.data::samples        0.028125
system.membus.snoop_filter.hitSingleRequests         0.028028
system.mem_ctrl.avgWrQLen                            0.028012
system.mem_ctrl.dram.totBusLat                       0.027967
system.cpu.dcache.ReadReq.mshrMissRate::total        0.021136
system.mem_ctrl.bytesWrittenSys                      0.019878
system.mem_ctrl.writeBursts                          0.018819
system.membus.pktCount::total                        0.018783
system.mem_ctrl.dram.bytesPerActivate::total         0.018777
system.cpu.dcache.ReadReq.mshrMisses::total          0.018715
system.mem_ctrl.requestorReadBytes::cpu.data         0.018713
system.m

## Modify features from the least important to the most important

In [19]:
importances = importances[importances>0]

In [20]:
importances

system.cpu.dcache.demandMissRate::total                5.963006e-02
system.cpu.dcache.overallMissRate::total               4.227520e-02
system.cpu.dcache.overallMissRate::cpu.data            4.184814e-02
system.cpu.dcache.demandMissRate::cpu.data             3.458669e-02
system.cpu.dcache.demandMshrMissRate::cpu.data         2.992976e-02
                                                           ...     
system.mem_ctrl.requestorReadAccesses::cpu.inst        2.571836e-07
system.cpu.dcache.writebacks::total                    1.343052e-07
system.cpu.dcache.overallMisses::total                 1.242419e-07
system.cpu.dcache.tags.avgOccs::total                  1.107723e-07
system.cpu.dcache.ReadReq.avgMshrMissLatency::total    9.824692e-08
Length: 343, dtype: float64

In [21]:
importance_list = importances.index.to_list()

In [22]:
attack_indices = y_test[y_test == 1].index[:100]
top_k_results = []

In [24]:
# 3. Tester l'ASR pour K allant de 1 à 100 (souvent le point de bascule est rapide)
for k in range(1, min(351, len(importance_list) + 1)):
    selected_features = importance_list[-k:]
    medians = X_train[y_train == 0][selected_features].median()
    
    success_count = 0
    for idx in attack_indices:
        x_adv = X_test.loc[idx].copy()
        x_adv[selected_features] = medians
        
        # Prédiction
        pred = rf.predict(pd.DataFrame([x_adv], columns=X_train.columns))[0]
        if pred == 0:
            success_count += 1
    
    asr = (success_count / len(attack_indices)) * 100
    top_k_results.append({"K": k, "ASR": asr, "Last_Feature": selected_features[0]})
    print(f"Top {-k} features: ASR = {asr:.2f}% (Dernière ajoutée: {selected_features[0]})")

# 4. Conversion en DataFrame pour analyse
df_top_k = pd.DataFrame(top_k_results)

Top -1 features: ASR = 0.00% (Dernière ajoutée: system.cpu.dcache.ReadReq.avgMshrMissLatency::total)
Top -2 features: ASR = 0.00% (Dernière ajoutée: system.cpu.dcache.tags.avgOccs::total)
Top -3 features: ASR = 0.00% (Dernière ajoutée: system.cpu.dcache.overallMisses::total)
Top -4 features: ASR = 0.00% (Dernière ajoutée: system.cpu.dcache.writebacks::total)
Top -5 features: ASR = 0.00% (Dernière ajoutée: system.mem_ctrl.requestorReadAccesses::cpu.inst)
Top -6 features: ASR = 0.00% (Dernière ajoutée: system.mem_ctrl.dram.dramBytesRead)
Top -7 features: ASR = 0.00% (Dernière ajoutée: system.cpu.dcache.ReadReq.misses::total)
Top -8 features: ASR = 0.00% (Dernière ajoutée: system.cpu.icache.overallMissRate::total)
Top -9 features: ASR = 0.00% (Dernière ajoutée: system.cpu.cpi)
Top -10 features: ASR = 0.00% (Dernière ajoutée: system.cpu.icache.overallMisses::cpu.inst)
Top -11 features: ASR = 0.00% (Dernière ajoutée: system.cpu.branchPred.btb.mispredict::DirectUncond)
Top -12 features: ASR 